# pandasの高速化

In [1]:
import numpy as np
import pandas as pd
from pandarallel import pandarallel

ここではorderbookのデータをDataFrameに読み込みます。

In [2]:
raw_df = pd.read_pickle("btcusd_2020-07-08.pickle")
raw_df.head()

,price,size,timestamp,side
2020-07-08 22:00:05,9433.24,2.305583,2020-07-08 22:00:05,bid
2020-07-08 22:00:05,9434.32,1.000000,2020-07-08 22:00:05,ask
2020-07-08 22:00:05,9434.35,0.530100,2020-07-08 22:00:05,ask
2020-07-08 22:00:05,9434.90,0.015848,2020-07-08 22:00:05,ask
2020-07-08 22:00:05,9435.00,0.050000,2020-07-08 22:00:05,ask


depth=10の板情報からbid/askが最も近い価格(price)と枚数(size)を抽出します。

In [3]:
bid = (
    raw_df.groupby("side")
    .get_group("bid")
    .groupby("timestamp")[["price", "size"]]
    .max()
)
ask = (
    raw_df.groupby("side")
    .get_group("ask")
    .groupby("timestamp")[["price", "size"]]
    .min()
)
df = pd.concat([bid, ask], axis=1)
df.index.name = None
df.columns = "bid_price", "bid_size", "ask_price", "ask_size"
df.head()

,bid_price,bid_size,ask_price,ask_size
2020-07-08 22:00:05,9433.24,2.305583,9433.25,0.015848
2020-07-08 22:00:06,9433.24,2.305583,9433.25,0.015848
2020-07-08 22:00:07,9433.24,2.323611,9433.25,0.015848
2020-07-08 22:00:08,9433.61,1.700000,9433.62,0.015848
2020-07-08 22:00:09,9433.51,1.650000,9433.52,0.015848


サンプルとして、priceとsizeから仲値を算出する関数を作成します。

In [4]:
def get_mid_price(bid, bid_sz, ask, ask_sz):
    try:
        mid_price = ask + (ask_sz / (ask_sz + bid_sz) * (bid - ask))
    except ZeroDivisionError:
        mid_price = None
    return mid_price


def get_mid_price_from_series(ser):
    return get_mid_price(*ser)


vfunc = np.vectorize(get_mid_price)

作成した関数を各行に対して `apply` メソッドで適用します。

In [5]:
%timeit df.apply(get_mid_price_from_series, axis=1)

37.5 ms ± 822 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


[numpy.vectorize](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html) はベクトル化した関数を定義します。引数には array-like なオブジェクトを渡します。

In [6]:
%timeit vfunc(*df.T.values)

1.79 ms ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


[Pandaral·lel](https://github.com/nalepae/pandarallel) はpandasの演算を利用可能なすべてのCPUで並列化するためのシンプルで効率的なツールです。 `initialize` メソッドで初期化して利用します。

In [7]:
pandarallel.initialize()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


DataFrameから `apply` メソッドの代わりに `parallel_apply` メソッドを実行すると、関数を並列に適用します。

In [8]:
%timeit df.parallel_apply(get_mid_price_from_series, axis=1)

54.7 ms ± 406 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
